# Stock Market Data Exploratory Analysis

In [78]:
import sys
sys.path.append('../src')

import polars as pl
import psycopg2
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from ipywidgets import Output
import IPython.display as display

## 1. Database Connection & Data Coverage

In [79]:
def get_connection():
    """Create database connection"""
    return psycopg2.connect(
        host="localhost",
        database="trading",
        user="postgres",
        password="trading123",
        port=5433,
    )

conn = get_connection()

coverage_query = """
    SELECT
        symbol,
        timeframe,
        MIN(timestamp) as earliest,
        MAX(timestamp) as latest,
        COUNT(*) as bar_count
    FROM ohlc
    GROUP BY symbol, timeframe
    ORDER BY symbol, timeframe
"""

coverage_df = pl.read_database(coverage_query, conn)
print("Data Coverage:")
print(coverage_df)

conn.close()

Data Coverage:
shape: (3, 5)
┌────────┬───────────┬─────────────────────────┬─────────────────────────┬───────────┐
│ symbol ┆ timeframe ┆ earliest                ┆ latest                  ┆ bar_count │
│ ---    ┆ ---       ┆ ---                     ┆ ---                     ┆ ---       │
│ str    ┆ str       ┆ datetime[μs, UTC]       ┆ datetime[μs, UTC]       ┆ i64       │
╞════════╪═══════════╪═════════════════════════╪═════════════════════════╪═══════════╡
│ DAX    ┆ 1D        ┆ 2024-02-19 00:00:00 UTC ┆ 2026-02-17 00:00:00 UTC ┆ 506       │
│ DAX    ┆ 1H        ┆ 2024-02-19 08:00:00 UTC ┆ 2026-02-17 13:00:00 UTC ┆ 4551      │
│ DAX    ┆ 3H        ┆ 2024-02-19 06:00:00 UTC ┆ 2026-02-17 12:00:00 UTC ┆ 2023      │
└────────┴───────────┴─────────────────────────┴─────────────────────────┴───────────┘


## 2. Load Data for Analysis

Load the last 2 years of daily and hourly data to identify patterns.

In [ ]:
def load_data(symbol="DAX", timeframe="1D", days=730):
    """Load OHLC data from PostgreSQL using polars"""
    conn = get_connection()
    
    query = """
        SELECT timestamp, open, high, low, close, volume
        FROM ohlc
        WHERE symbol = %s
          AND timeframe = %s
          AND timestamp > NOW() - INTERVAL '%s days'
        ORDER BY timestamp
    """
    
    df = pl.read_database(query, conn, execute_options={"parameters": (symbol, timeframe, days)})
    conn.close()
    
    return df

daily_df = load_data("DAX", "1D", days=730)
print(f"\nLoaded {len(daily_df)} daily bars")
print(daily_df.head())
print(f"\nDate range: {daily_df['timestamp'].min()} to {daily_df['timestamp'].max()}")


Loaded 506 daily bars
shape: (5, 6)
┌─────────────────────────┬───────────────┬───────────────┬───────────────┬───────────────┬────────┐
│ timestamp               ┆ open          ┆ high          ┆ low           ┆ close         ┆ volume │
│ ---                     ┆ ---           ┆ ---           ┆ ---           ┆ ---           ┆ ---    │
│ datetime[μs, UTC]       ┆ decimal[38,5] ┆ decimal[38,5] ┆ decimal[38,5] ┆ decimal[38,5] ┆ i64    │
╞═════════════════════════╪═══════════════╪═══════════════╪═══════════════╪═══════════════╪════════╡
│ 2024-02-19 00:00:00 UTC ┆ 17075.74000   ┆ 17098.61000   ┆ 17049.22000   ┆ 17092.26000   ┆ 0      │
│ 2024-02-20 00:00:00 UTC ┆ 17066.23000   ┆ 17084.87000   ┆ 17019.15000   ┆ 17068.43000   ┆ 0      │
│ 2024-02-21 00:00:00 UTC ┆ 17102.08000   ┆ 17157.95000   ┆ 17063.25000   ┆ 17118.12000   ┆ 0      │
│ 2024-02-22 00:00:00 UTC ┆ 17326.76000   ┆ 17429.66000   ┆ 17279.73000   ┆ 17370.45000   ┆ 0      │
│ 2024-02-23 00:00:00 UTC ┆ 17372.01000   ┆ 17443.7400

## 3. Data Quality Checks

In [ ]:
print("Missing values:")
print(daily_df.null_count())

daily_df_with_checks = daily_df.with_columns([
    ((pl.col('high') - pl.col('low')) / pl.col('low') * 100).alias('range_pct'),
    (pl.col('high') >= pl.col('low')).alias('valid_hl'),
    (pl.col('high') >= pl.col('open')).alias('valid_ho'),
    (pl.col('high') >= pl.col('close')).alias('valid_hc'),
    (pl.col('low') <= pl.col('open')).alias('valid_lo'),
    (pl.col('low') <= pl.col('close')).alias('valid_lc'),
])

invalid_bars = daily_df_with_checks.filter(
    ~pl.col('valid_hl') | ~pl.col('valid_ho') | ~pl.col('valid_hc') | 
    ~pl.col('valid_lo') | ~pl.col('valid_lc')
)

print(f"\nInvalid bars found: {len(invalid_bars)}")
if len(invalid_bars) > 0:
    print(invalid_bars)

print("\n Top 10 largest daily moves:")
print(daily_df_with_checks.sort('range_pct', descending=True).select(['timestamp', 'open', 'high', 'low', 'close', 'range_pct']).head(10))

Missing values:
shape: (1, 6)
┌───────────┬──────┬──────┬─────┬───────┬────────┐
│ timestamp ┆ open ┆ high ┆ low ┆ close ┆ volume │
│ ---       ┆ ---  ┆ ---  ┆ --- ┆ ---   ┆ ---    │
│ u32       ┆ u32  ┆ u32  ┆ u32 ┆ u32   ┆ u32    │
╞═══════════╪══════╪══════╪═════╪═══════╪════════╡
│ 0         ┆ 0    ┆ 0    ┆ 0   ┆ 0     ┆ 0      │
└───────────┴──────┴──────┴─────┴───────┴────────┘

Invalid bars found: 0

 Top 10 largest daily moves:
shape: (10, 6)
┌──────────────┬───────────────┬───────────────┬───────────────┬───────────────┬───────────────┐
│ timestamp    ┆ open          ┆ high          ┆ low           ┆ close         ┆ range_pct     │
│ ---          ┆ ---           ┆ ---           ┆ ---           ┆ ---           ┆ ---           │
│ datetime[μs, ┆ decimal[38,5] ┆ decimal[38,5] ┆ decimal[38,5] ┆ decimal[38,5] ┆ decimal[38,5] │
│ UTC]         ┆               ┆               ┆               ┆               ┆               │
╞══════════════╪═══════════════╪═══════════════╪════════════

## 4. Interactive Candlestick Charts

Use Plotly for interactive exploration of price action.

In [ ]:
def plot_candlestick(df, title="DAX Candlestick Chart", show_volume=True):
    """
    Create interactive candlestick chart with Plotly
    
    Args:
        df: Polars DataFrame with columns: timestamp, open, high, low, close, volume
        title: Chart title
        show_volume: Whether to show volume subplot
    """
    if show_volume:
        fig = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.03,
            row_heights=[0.7, 0.3],
            subplot_titles=(title, 'Volume')
        )
    else:
        fig = go.Figure()
    
    price_min = float(df['low'].min())
    price_max = float(df['high'].max())
    
    y_range = [price_min + i * (price_max - price_min) / 100 for i in range(101)]
    x_values = df['timestamp'].to_list() * len(y_range)
    y_values = [y for y in y_range for _ in range(len(df))]
    
    price_overlay = go.Scatter(
        x=x_values,
        y=y_values,
        mode='markers',
        marker=dict(size=0.1, color='rgba(0,0,0,0)'),
        showlegend=False,
        hovertemplate='<b>Price: %{y:,.2f}</b><extra></extra>',
        name='Price Level'
    )
    
    candlestick = go.Candlestick(
        x=df['timestamp'].to_list(),
        open=df['open'].to_list(),
        high=df['high'].to_list(),
        low=df['low'].to_list(),
        close=df['close'].to_list(),
        name='OHLC',
        hovertext=[
            f"<b>Price Levels:</b><br>" +
            f"Open: {o:.2f}<br>" +
            f"High: <b>{h:.2f}</b><br>" +
            f"Low: <b>{l:.2f}</b><br>" +
            f"Close: {c:.2f}<br>" +
            f"Range: {h-l:.2f} ({(h-l)/l*100:.2f}%)"
            for o, h, l, c in zip(
                df['open'].to_list(),
                df['high'].to_list(),
                df['low'].to_list(),
                df['close'].to_list()
            )
        ],
        hoverinfo='text+x',
    )
    
    if show_volume:
        fig.add_trace(price_overlay, row=1, col=1)
        fig.add_trace(candlestick, row=1, col=1)
        
        colors = ['red' if df['close'][i] < df['open'][i] else 'green' 
                  for i in range(len(df))]
        
        volume_bars = go.Bar(
            x=df['timestamp'].to_list(),
            y=df['volume'].to_list(),
            name='Volume',
            marker_color=colors,
            showlegend=False
        )
        fig.add_trace(volume_bars, row=2, col=1)
    else:
        fig.add_trace(price_overlay)
        fig.add_trace(candlestick)
        fig.update_layout(title=title)
    
    fig.update_layout(
        xaxis_rangeslider_visible=False,
        height=800,
        hovermode='closest',
        template='plotly_dark',
    )
    
    fig.update_xaxes(
        title_text="Date",
        row=2 if show_volume else 1,
        col=1,
        showspikes=True,
        spikemode='across',
        spikesnap='cursor',
        spikedash='dot',
        spikecolor='rgba(150, 150, 150, 0.5)',
        spikethickness=1
    )
    
    fig.update_yaxes(
        title_text="Price",
        row=1,
        col=1,
        showspikes=True,
        spikemode='across+toaxis+marker',
        spikesnap='cursor',
        spikedash='solid',
        spikecolor='rgba(255, 255, 255, 0.8)',
        spikethickness=2,
        showgrid=True,
        gridcolor='rgba(100, 100, 100, 0.2)',
    )
    
    if show_volume:
        fig.update_yaxes(title_text="Volume", row=2, col=1)
    
    return fig

recent_daily = load_data("DAX", "1D", days=180)
fig = plot_candlestick(recent_daily, "DAX Daily - Last 6 Months")
fig.show()

## 5. Multi-Timeframe Analysis

Compare daily, 3H, and 1H charts to understand how patterns look across timeframes.

In [ ]:
hourly_1h = load_data("DAX", "1H", days=60)
hourly_3h = load_data("DAX", "3H", days=60)

print(f"1H bars: {len(hourly_1h)}")
print(f"3H bars: {len(hourly_3h)}")

recent_3h = load_data("DAX", "3H", days=30)
fig_3h = plot_candlestick(recent_3h, "DAX 3H - Last 30 Days")
fig_3h.show()

1H bars: 330
3H bars: 147


## 6. Manual Pattern Identification

In [84]:
def plot_date_range(symbol="DAX", timeframe="1H", start_date=None, end_date=None):
    """
    Plot a specific date range for detailed pattern inspection
    
    Example:
        plot_date_range("DAX", "1H", "2024-01-15", "2024-01-25")
    """
    conn = get_connection()
    
    query = """
        SELECT timestamp, open, high, low, close, volume
        FROM ohlc
        WHERE symbol = %s
          AND timeframe = %s
          AND timestamp BETWEEN %s AND %s
        ORDER BY timestamp
    """
    
    df = pl.read_database(
        query, 
        conn, 
        execute_options={"parameters": (symbol, timeframe, start_date, end_date)}
    )
    conn.close()
    
    title = f"{symbol} {timeframe} - {start_date} to {end_date}"
    fig = plot_candlestick(df, title)
    return fig

fig = plot_date_range("DAX", "1H", "2024-11-01", "2024-11-30")
fig.show()

## 7. Label Data

Create a labeled dataset of manually identified patterns.

In [ ]:
labeled_setups = [
    # {
    # },
]

if len(labeled_setups) > 0:
    setups_df = pl.DataFrame(labeled_setups)
    print(f"Labeled {len(setups_df)} setups")
    print(setups_df)
else:
    print("No setups labeled yet.")

No setups labeled yet.


## 8. Save Labeled Dataset to CSV

In [86]:
if len(labeled_setups) > 0:
    setups_df = pl.DataFrame(labeled_setups)
    output_path = '../data/wm_setups_labeled.csv'
    setups_df.write_csv(output_path)
    print(f"Saved {len(setups_df)} labeled setups to {output_path}")
else:
    print("No setups to save.")

No setups to save.
